# Init

In [10]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [11]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [12]:
conn=init_connection(dsn)

2021-07-24 13:49:00.461 INFO    root: Connecting to the PostgreSQL database...
2021-07-24 13:49:00.486 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-07-24 13:49:00.488 INFO    root: PostgreSQL connection status: 0
2021-07-24 13:49:00.490 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


# 23/7/2021

In [17]:
from enum import Enum,IntEnum
from typing import NamedTuple

In [18]:
class Detection(NamedTuple):
    name: str
    prob: float

In [19]:
detection=Detection('car',0.95)

In [35]:
detection.name,detection.prob
detection

Detection(name='car', prob=0.95)

In [37]:
import numpy as np
x=np.array([1,2,3])

In [39]:
np.asarray(x)

array([1, 2, 3])

# 24/7/2021

In [13]:
query_project_SQL="""
                    SELECT
                        p.id,
                            p.name,
                            description,
                            deployment_id,
                            project_path,
                            dt.name as deployment_type
                    FROM
                        public.project p
                        LEFT JOIN deployment_type dt ON dt.id = p.deployment_id
                    WHERE
                        p.id = 2;"""

In [15]:
db_fetchone(query_project_SQL,conn)

Record(id=2, name='First Project', description='This project is for object detection', deployment_id=2, project_path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/project', deployment_type='Object Detection with Bounding Boxes')

### Get column names for DB

In [47]:
column_names=[]
data_rows=[]
with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute("""SELECT
    d.id AS dataset_id,
    d.name AS dataset_name,
    d.dataset_size,
    pd.updated_at 
FROM
    public.project_dataset pd
    LEFT JOIN public.dataset d ON d.id = pd.dataset_id
WHERE
    pd.project_id = 7;""")
                column_names = [desc[0] for desc in cur.description]
                for row in cur:
                    data_rows.append(row)

print("Column names: {}\n".format(column_names))

Column names: ['dataset_id', 'dataset_name', 'dataset_size', 'updated_at']



In [42]:
from psycopg2.extras import NamedTupleCursor
with conn.cursor(cursor_factory=NamedTupleCursor) as cur:
   cur.execute("""
   SELECT
    d.id AS dataset_id,
    d.name AS dataset_name,
    d.dataset_size,
    pd.updated_at 
FROM
    public.project_dataset pd
    LEFT JOIN public.dataset d ON d.id = pd.dataset_id
WHERE
    pd.project_id = 7;""")
   return_all=cur.fetchall()

In [43]:
for i in return_all:
    display(i)

Record(dataset_id=2, dataset_name='My Second Dataset', dataset_size=4, updated_at=datetime.datetime(2021, 7, 24, 14, 25, 45, 68588, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))

Record(dataset_id=4, dataset_name='My Fourth Dataset', dataset_size=4, updated_at=datetime.datetime(2021, 7, 24, 14, 25, 45, 68588, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))

In [44]:
import pandas as pd

####  Create Pandas Dataframe using PostgreSQL column names

In [48]:
df=pd.DataFrame(return_all,columns=column_names)
display(df)

,dataset_id,dataset_name,dataset_size,updated_at
0,2,My Second Dataset,4,2021-07-24 14:25:45.068588+08:00
1,4,My Fourth Dataset,4,2021-07-24 14:25:45.068588+08:00
